[ratsgo's blog](https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/06/01/LDA/)  
[EnGear](http://www.engear.net/wp/topic-modeling-gensimpython/)  
[컴언-Ngram](http://blog.ilkyu.kr/entry/%EC%96%B8%EC%96%B4-%EB%AA%A8%EB%8D%B8%EB%A7%81-ngram)  
[LOVIT x DATA SCIENCE-lemmatization](https://lovit.github.io/nlp/2018/06/07/lemmatizer/)  
하기 내용은 상기 링크를 정리한것.

# Topic Modeling(Latent dirichlet Allocation)

In [1]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Andrew\AppData\L
[nltk_data]     ocal\conda\conda\envs\text3.5\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# !python -m spacy download en

In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [2]:
# gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim # don't skip this
import matplotlib.pyplot as plt

%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level = logging.ERROR)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\Andrew\AppData\Local\conda\conda\envs\text3.5\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
# 불용단어 로딩
# NLTK stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [4]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.tail()

['rec.autos' 'comp.sys.mac.hardware' 'rec.motorcycles' 'misc.forsale'
 'comp.os.ms-windows.misc' 'alt.atheism' 'comp.graphics'
 'rec.sport.baseball' 'rec.sport.hockey' 'sci.electronics' 'sci.space'
 'talk.politics.misc' 'sci.med' 'talk.politics.mideast'
 'soc.religion.christian' 'comp.windows.x' 'comp.sys.ibm.pc.hardware'
 'talk.politics.guns' 'talk.religion.misc' 'sci.crypt']


,content,target,target_names
9995,From: sanjay@kin.lap.upenn.edu (Sanjay Sinha)\...,8,rec.motorcycles
9996,Organization: Central Michigan University\nFro...,7,rec.autos
9997,From: cr292@cleveland.Freenet.Edu (Jim Schenk)...,10,rec.sport.hockey
9998,From: wbdst+@pitt.edu (William B Dwinnell)\nSu...,1,comp.graphics
9999,From: ehung@ampex.com (Eric Hung)\nSubject: Re...,3,comp.sys.ibm.pc.hardware


In [5]:
# Convert to list
data = df.content.values.tolist()
# type(data)
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters(공백제거)
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


## 단어 토큰화(형태소 분할)

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) # deacc = True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


## Bigram과 Trigram모델 만들기

In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

C:\Users\Andrew\AppData\Local\conda\conda\envs\text3.5\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [8]:
bigram['car']

C:\Users\Andrew\AppData\Local\conda\conda\envs\text3.5\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['c', 'a', 'r']

In [9]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [10]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


## 불용단어 제거

In [11]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [12]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [13]:
def make_trigram(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

### 단어 원형 복원(lemmatization)
> studies = studi + es  
studying = study + ing

In [14]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """
    https://spacy.io/api/annotation
    """
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

함수 실행 파트

In [15]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do  lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [16]:
print(data_lemmatized[:1])

[['where', 's', 'thing', 'car', 'nntp_post', 'host', 'rac_wam', 'umd', 'organization', 'university', 'maryland_college', 'park', 'line', 'wonder', 'anyone', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'whatev', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


## 토픽 모델에 필요한 사전과 코퍼스 생성하기

LDA의 주요 입력값 코드를 생성할 것인데 그 두 가지 값은 사전(id2word)과 copus이다.

In [17]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

In [18]:
id2word

In [19]:
# texts

In [20]:
id2word.doc2bow(texts[0])

[(0, 1),
 (1, 2),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 5),
 (7, 1),
 (8, 1),
 (9, 2),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 2),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1)]

In [21]:
type(id2word.doc2bow(texts[0]))

list

In [22]:
type(id2word.doc2bow(texts[0])[0])

tuple

In [23]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [24]:
# view
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 5), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1)]]


Gensim은 문서의 각 단어들마다 유니크한 ID를 생성한다.  
위의 튜플은 (word_id, word_frequency)이 매핑되어 있다.  
이것이 LDA모델의 입력으로 사용된다.  
아래는 아이디에 따른 단어를 확인해볼 수 있다.

In [25]:
id2word[0]

'addition'

또는 사람이 이해 가능한 형태의 코퍼스를 생성할 수 있다.   
(단어, 빈도 수)

In [26]:
# Human readable format of corpus (term-frequency)

[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('anyone', 2),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('front_bumper', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('maryland_college', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_post', 1),
  ('organization', 1),
  ('park', 1),
  ('production', 1),
  ('rac_wam', 1),
  ('really', 1),
  ('rest', 1),
  ('s', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('umd', 1),
  ('university', 1),
  ('whatev', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

## 토픽 모델 생성

LDA모델 훈련을 위해서는 Copus, 사전, 토픽 개수를 제공해야한다.  
그 외에도, 알파와 에타는 토픽의 희소성에 영향을 주는 하이퍼파라미터 이다.  
Gensim docs에 따르면 기본값은 모두 '1.0/num_topics'을 기본값으로 사용한다.  <br>
<br>
chunksize는 각 훈련 chunk에서 사용할 문서의 수이다.   
update_every는 모델 매개변수를 업데이트 해야하는 빈도를 결정하고,  
passes는 총 훈련 과정 수를 결정한다.  

https://radimrehurek.com/gensim/models/ldamodel.html

In [27]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word, 
                                            num_topics = 20, 
                                            random_state = 100, 
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

## LDA 모델의 토픽 보기
위의 LDA모델은 각 토픽이 키워드의 조합이고,  
각 키워드가 토픽에 일정한 가중치를 부여하는 20개의 주제로 구성된다.  
`lda_model.print_topics()`를 사용하여 각 토픽의 키워드와 각 키워드의 중요도(가중치)를 볼 수 있다.

In [28]:
# Print the keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.034*"_" + 0.029*"blue" + 0.021*"tank" + 0.014*"cubs_suck" + 0.012*"eg" + '
  '0.012*"gas" + 0.011*"hi" + 0.007*"henry_spencer" + 0.007*"cigarette" + '
  '0.006*"xlib"'),
 (1,
  '0.020*"value" + 0.020*"bus" + 0.019*"specifically" + 0.015*"function" + '
  '0.015*"associate" + 0.013*"motorcycle" + 0.011*"properly" + 0.010*"code" + '
  '0.008*"confuse" + 0.008*"error"'),
 (2,
  '0.023*"window" + 0.020*"card" + 0.017*"file" + 0.014*"drive" + 0.013*"use" '
  '+ 0.013*"system" + 0.011*"problem" + 0.010*"run" + 0.009*"color" + '
  '0.009*"do"'),
 (3,
  '0.035*"game" + 0.033*"team" + 0.019*"player" + 0.017*"play" + 0.017*"win" + '
  '0.016*"hockey" + 0.013*"season" + 0.011*"contact" + 0.011*"year" + '
  '0.010*"nhl"'),
 (4,
  '0.026*"wire" + 0.017*"circuit" + 0.015*"faq" + 0.015*"connect" + '
  '0.013*"wiring" + 0.013*"voice" + 0.013*"cover" + 0.011*"outlet" + '
  '0.011*"neutral" + 0.010*"conference"'),
 (5,
  '0.030*"government" + 0.024*"gun" + 0.022*"law" + 0.017*"state" + '
  '0.0

## 해석 방법  
```
[(0,
  '0.034*"_" + 0.029*"blue" + 0.021*"tank" + 0.014*"cubs_suck" + 0.012*"eg" + '
  '0.012*"gas" + 0.011*"hi" + 0.007*"henry_spencer" + 0.007*"cigarette" + '
  '0.006*"xlib"'),
```
토픽0 는 위와 같이 표현된다.  
이 토픽에 대해 기여하고 있는 상위 10개 키워드는 'blue', 'tank', 'cubs_suck'등 이며,  
토픽 0에 대한 'blue'의 가중치는 0.029이다.  
가중치는 키워드가 해당 토픽에 얼마나 중요하는지를 나타낸다.  
다시말하자면 토픽 0는 blue나 tank일 수 있다.  

## 모델 난이도와 일관성 점수 계산

In [29]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts = data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.754130909406506

Coherence Score:  0.5152202258019667


## 토픽-키워드 시각화 

In [30]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
7      16.262438        1       1  0.337009 -0.001268
13     15.228902        1       2  0.291093 -0.003100
18     14.564972        1       3  0.328797 -0.128176
2       6.162430        1       4  0.091721  0.226476
15      5.974024        1       5  0.059664  0.222616
10      4.344128        1       6 -0.084991 -0.112872
17      4.177711        1       7  0.060997 -0.182966
12      3.586029        1       8 -0.022525 -0.036449
5       3.366010        1       9 -0.022898 -0.021223
16      3.271635        1      10 -0.029311  0.219728
6       3.227684        1      11 -0.062714 -0.017222
9       2.969382        1      12 -0.062883  0.139502
14      2.741264        1      13 -0.088857 -0.062628
3       2.731828        1      14 -0.047746 -0.117471
8       2.290905        1      15 -0.111489  0.003679
19      2.124099        1      16 -0.113660 -0.023379
11      2.081701        1      17 -0.092185 -0.110817
4       1.693961        1      18 -0.134208  0.001595
0       1.679178        1      19 -0.164194 -0.028011
1       1.521725        1      20 -0.131619  0.031985, topic_info=      Category          Freq          Term         Total  loglift  logprob
term                                                                      
45019  Default  57084.000000            ax  57084.000000  30.0000  30.0000
17409  Default  12491.000000          line  12491.000000  29.0000  29.0000
34840  Default  16294.000000           not  16294.000000  28.0000  28.0000
27874  Default  11141.000000  organization  11141.000000  27.0000  27.0000
52486  Default   3845.000000           max   3845.000000  26.0000  26.0000
632    Default  10454.000000            do  10454.000000  25.0000  25.0000
4050   Default   9655.000000         write   9655.000000  24.0000  24.0000
28197  Default   7586.000000       article   7586.000000  23.0000  23.0000
58112  Default   8048.000000            be   8048.000000  22.0000  22.0000
43932  Default  11783.000000         would  11783.000000  21.0000  21.0000
70641  Default   6906.000000           say   6906.000000  20.0000  20.0000
36245  Default   5804.000000            go   5804.000000  19.0000  19.0000
53481  Default   4407.000000     nntp_post   4407.000000  18.0000  18.0000
14393  Default   4083.000000          host   4083.000000  17.0000  17.0000
38035  Default   6852.000000           get   6852.000000  16.0000  16.0000
17108  Default   5027.000000    university   5027.000000  15.0000  15.0000
3146   Default   1863.000000         space   1863.000000  14.0000  14.0000
18760  Default   2294.000000        window   2294.000000  13.0000  13.0000
22731  Default   3735.000000             s   3735.000000  12.0000  12.0000
37395  Default   7292.000000          know   7292.000000  11.0000  11.0000
53402  Default   3461.000000          year   3461.000000  10.0000  10.0000
59065  Default   4598.000000           use   4598.000000   9.0000   9.0000
59995  Default   1540.000000          game   1540.000000   8.0000   8.0000
63374  Default   1948.000000          mail   1948.000000   7.0000   7.0000
71267  Default   3006.000000         right   3006.000000   6.0000   6.0000
66387  Default   1685.000000     christian   1685.000000   5.0000   5.0000
48690  Default   1746.000000    government   1746.000000   4.0000   4.0000
57776  Default   1464.000000          team   1464.000000   3.0000   3.0000
45542  Default   3380.000000        system   3380.000000   2.0000   2.0000
19607  Default   2012.000000          card   2012.000000   1.0000   1.0000
...        ...           ...           ...           ...      ...      ...
25573  Topic20    119.371460          guru    120.323341   4.1774  -5.3259
32683  Topic20    117.623947  countersteer    118.575829   4.1773  -5.3407
28971  Topic20    113.026604       bradley    113.978485   4.1769  -5.3805
56511  Topic20    112.217239          lane    113.169121   4.1769  -5

원이 클수록 그 토픽이 더 일반적이다.  
즉, 특성화 하지 않고 여러 단어가 등장한다는 의미이다.  
좋은 토픽모델은 하나의 사분면에 모두 군집화 되지 않고 차트 전체적으로 흩어진 상태가 가장 좋다.

## 한글적용

In [41]:
from bs4 import BeautifulSoup
import urllib.request
from konlpy.tag import Twitter
from konlpy.utils import pprint
from collections import Counter
# import konlpy
import re

In [42]:
def get_text(URL):
    text = []
#     URL = ''
    for i in range(10):
        URL2 = URL + str(3956688 + i)
        print(URL2)
        source_code_from_URL = urllib.request.urlopen(URL2)
        soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')

        for t in soup.find_all('div', id="articleBodyContents"):
            textin = str(t.find_all(text=True))
            text.append(textin)
#         URL = ""
    return text

In [43]:
twitter = Twitter()

In [44]:
URL = 'http://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=732&oid=015&aid=000'

In [45]:
kor_result= get_text(URL)

http://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=732&oid=015&aid=0003956688
http://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=732&oid=015&aid=0003956689
http://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=732&oid=015&aid=0003956690
http://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=732&oid=015&aid=0003956691
http://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=732&oid=015&aid=0003956692
http://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=732&oid=015&aid=0003956693
http://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=732&oid=015&aid=0003956694
http://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=732&oid=015&aid=0003956695
http://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=732&oid=015&aid=0003956696
http://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=732&oid=015&aid=0003956697


In [46]:
kor_result2 = []
for i in kor_result:
    rep = i.replace("본문", "").replace('내용', '').replace('한국경제','').replace('모바일한경 구독신청', '')
    rep = rep.replace('[\\n,  , \\n,  TV플레이어 , \\n,  // TV플레이어 , \\n, \\n// flash 오류를 우회하기 위한 함수 추가\\nfunction _flash_removeCallback() {}\\n, \\n, ','')
        # Remove Emails
    rep = re.sub('\S*@\S*\s?', '', rep)

    # Remove new line characters(공백제거)
    rep = re.sub('\s+', ' ', rep)

    # Remove distracting single quotes
    rep = re.sub("\'", "", rep)
    kor_result2.append(rep)

In [47]:
kor_result2

['[\\n,  , \\n,  TV플레이어 , \\n,  // TV플레이어 , \\n, \\n// flash 오류를 우회하기 위한 함수 추가\\nfunction _flash_removeCallback() {}\\n, \\n, "유니버시티 블록체인 리서치 이니셔티브 프로젝트 수행", 고려대 본관. / 사진=한경 DB, 고려대가 글로벌 블록체인 기업 리플(Ripple)과 파트너십을 체결해 관련 전문인력 양성과 연구 프로젝트 수행에 나선다., 리플은 블록체인을 이용해 저렴하면서 빠른 송금 서비스를 제공하는 회사로 자체 암호토큰 XRP를 이용한 서비스도 제공한다., 7일 고려대에 따르면 이 대학 정보보호대학원 블록체인보안연구센터는 지난 4일 리플과 장기 연구 프로젝트 ‘유니버시티 블록체인 리서치 이니셔티브(University Blockchain Research Initiative)’ 수행을 위한 파트너십을 맺었다., 세계 유수의 대학들과 함께 블록체인 및 암호화폐 관련 학문적·기술적 연구를 수행하도록 지원하는 연구 프로젝트다. 이 프로젝트 수행을 위해 리플과 파트너십을 맺은 대학은 MIT(매사추세츠공대) 프린스턴대 스탠퍼드대 등 전세계 17곳으로 국내에서는 고려대가 유일하다., 파트너십 체결에 따라 고려대 블록체인보안연구센터는 블록체인 합의 알고리즘의 안전성, 스마트 콘트랙트(smart contract)의 안전성 등 블록체인 및 암호화폐에 필요한 보안 기술을 중점적으로 연구하며 리플은 연구 수행에 필요한 재정적·기술적 지원을 제공한다., 아울러 사회적으로 부족한 블록체인 전문가를 양성하는 효과도 있을 것으로 예상된다., 에릭 반 밀텐버그 리플 비즈니스 운영 수석부사장은 “‘유니버시티 블록체인 리서치 이니셔티브’는 대학이 블록체인 기술의 혁신에 있어 중요 역할을 할 수 있게 하는 디딤돌이 될 것”이라며 “블록체인 관련 산업체에서 일할 수 있는 인력을 길러내는 효과도 있을 것”이라고 말했다., 이상진 고려대 정보보호대학원장도 “이번 파트너십 체결과 프로젝트 수행으로 리플

In [48]:
ko_copus = []
only_nouns = []
mapping = {}
for i in kor_result2:
    make_ko = twitter.nouns(i)
#     print(make_ko)
    kor_count = Counter(make_ko)
    mid_list = []
    mid_nouns_list = []
    for idx, key in enumerate(kor_count):
        mid_list.append((idx, kor_count[key]))
        mid_nouns_list.append(key)
        mapping[idx] = key
#         print([(idx, kor_count[key])])
    only_nouns.append(mid_nouns_list)
    ko_copus.append(mid_list)

In [49]:
# only_nouns

In [50]:
id2word = corpora.Dictionary(only_nouns)

In [51]:
id2word

In [52]:
ko_copus[:1]

[[(0, 4),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 3),
  (6, 1),
  (7, 2),
  (8, 1),
  (9, 6),
  (10, 2),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 6),
  (15, 14),
  (16, 1),
  (17, 7),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 4),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 2),
  (28, 1),
  (29, 1),
  (30, 5),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 2),
  (38, 1),
  (39, 1),
  (40, 5),
  (41, 1),
  (42, 1),
  (43, 3),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 4),
  (50, 3),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 3),
  (62, 1),
  (63, 2),
  (64, 2),
  (65, 14),
  (66, 2),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 2),
  (71, 2),
  (72, 1),
  (73, 1),
  (74, 7),
  (75, 1),
  (76, 2),
  (77, 2),
  (78, 2),
  (79, 3),
  (80, 1),
  (81, 1),
  (82, 3),
  (83, 1),
  (84, 1),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 1),
  (89, 1),
  (90, 1),
  (91, 

In [53]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=ko_copus,
                                            id2word=id2word, 
                                            num_topics = 20, 
                                            random_state = 100, 
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

In [54]:
pprint(lda_model.print_topics())
doc_lda = lda_model[ko_copus]

[(0,
  '0.004*"량" + 0.004*"대폭" + 0.004*"상위" + 0.004*"살짝" + 0.004*"미래에셋" + '
  '0.004*"목록" + 0.004*"루스" + 0.004*"시간" + 0.004*"랭킹" + 0.004*"흐름"'),
 (1,
  '0.004*"량" + 0.004*"대폭" + 0.004*"상위" + 0.004*"살짝" + 0.004*"미래에셋" + '
  '0.004*"목록" + 0.004*"루스" + 0.004*"시간" + 0.004*"랭킹" + 0.004*"흐름"'),
 (2,
  '0.004*"재" + 0.004*"산" + 0.004*"기술" + 0.004*"블록" + 0.004*"사진" + 0.004*"장기" + '
  '0.004*"운영" + 0.004*"전세계" + 0.004*"등" + 0.004*"이니셔티브"'),
 (3,
  '0.021*"시티" + 0.018*"이니셔티브" + 0.018*"기술" + 0.017*"고려대" + 0.016*"버그" + '
  '0.016*"역할" + 0.015*"산" + 0.015*"유일하" + 0.014*"본관" + 0.014*"블록"'),
 (4,
  '0.004*"량" + 0.004*"대폭" + 0.004*"상위" + 0.004*"살짝" + 0.004*"미래에셋" + '
  '0.004*"목록" + 0.004*"루스" + 0.004*"시간" + 0.004*"랭킹" + 0.004*"흐름"'),
 (5,
  '0.006*"역할" + 0.006*"시티" + 0.005*"본관" + 0.005*"일" + 0.005*"스탠퍼드대" + '
  '0.005*"유일하" + 0.005*"리서치" + 0.005*"이번" + 0.005*"버그" + 0.004*"바로가기"'),
 (6,
  '0.005*"이니셔티브" + 0.005*"이" + 0.005*"기술" + 0.005*"봉구" + 0.005*"고려대" + '
  '0.004*"버그" + 0.004*"일" + 0.004*"무단" + 0.0

In [55]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(ko_copus)) # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts = only_nouns, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.10038842709372

Coherence Score:  0.8596839625612963


In [56]:
mapping[2]

'케이'

In [1]:
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, ko_copus, id2word)
# vis